<a href="https://colab.research.google.com/github/dangduong2806/google-colab/blob/main/Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving colors1.csv to colors1.csv


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import pandas as pd
import transformers
from tensorflow.keras.layers import Input, Dense, Lambda

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("colors1.csv")
df.head()

,name,red,green,blue
0,parakeet,174,182,87
1,saddle brown,88,52,1
2,cucumber crush,222,237,215
3,pool blue,134,194,201
4,distance,98,110,130


Standardize data

In [ ]:
df["red"] = df["red"] / 255
df["green"] = df["green"] / 255
df["blue"] = df["blue"] / 255
df.head()

,name,red,green,blue
0,parakeet,0.682353,0.713725,0.341176
1,saddle brown,0.345098,0.203922,0.003922
2,cucumber crush,0.870588,0.929412,0.843137
3,pool blue,0.525490,0.760784,0.788235
4,distance,0.384314,0.431373,0.509804


In [ ]:
len(df["name"])

14357

Tách input và output

In [ ]:
X_texts = df['name'].astype(str).values
y_rgb = df[['red', 'green', 'blue']].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Tách train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_texts, y_rgb, test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_text(texts):
    texts = list(texts)
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "token_type_ids": encodings["token_type_ids"]
    }

In [ ]:
from transformers import TFBertModel

In [ ]:
def create_bert_to_rgb_model():
    bert = TFBertModel.from_pretrained("bert-base-uncased")
    for layer in bert.layers:
        layer.trainable = False  # freeze toàn bộ BERT

    input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

    # # Fix: Add output_shape parameter
    # bert_outputs = tf.keras.layers.Lambda(
    #     lambda x: bert(input_ids=x[0], attention_mask=x[1]),
    #     output_shape=lambda input_shape: (input_shape[0][0], input_shape[0][1], 768)  # BERT base has 768 hidden size
    # )([input_ids, attention_mask])

    # # Fix: Extract CLS token with output_shape
    # cls_token = tf.keras.layers.Lambda(
    #     lambda x: x.last_hidden_state[:, 0, :],
    #     output_shape=(768,)  # CLS token is 768-dimensional for BERT base
    # )(bert_outputs)

    # Combined Lambda layer that does both BERT call and CLS extraction
    cls_token = tf.keras.layers.Lambda(
        lambda x: bert(input_ids=x[0], attention_mask=x[1]).last_hidden_state[:, 0, :],
        output_shape=(768,)
    )([input_ids, attention_mask])

    # Chỉ train lớp này
    x = tf.keras.layers.Dense(128, activation="relu")(cls_token)
    x = tf.keras.layers.Dense(3, activation="sigmoid")(x)  # sigmoid vì RGB chuẩn hóa [0, 1]

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=x)
    return model

In [ ]:
model = create_bert_to_rgb_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="mse",
    metrics=["mae", "mse"]
)

In [ ]:
train_tokens = tokenize_text(X_train)
val_tokens = tokenize_text(X_val)

In [ ]:
history = model.fit(
    train_tokens,
    y_train,
    validation_data=(val_tokens, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)


NameError: name 'model' is not defined